In [15]:
from __future__ import print_function

# We'll need numpy for some mathematical operations
import numpy as np
import copy

# matplotlib for displaying the output
import matplotlib.pyplot as plt
import matplotlib.style as ms
ms.use('seaborn-muted')
%matplotlib inline

# and IPython.display for audio output
import IPython.display

# Librosa for audio
import librosa
import librosa.display
import pandas as pd
import tensorflow as tf
import timeit
import pickle 

from joblib import Parallel, delayed
import multiprocessing
import os
import time

num_cores = multiprocessing.cpu_count()
print(num_cores)
    



8


In [16]:

samples_hyper_parameters = [
    { 'bank': 'grand piano', 'audio_file': 'piano_test.aiff', 'data_file':'piano_test.txt'},
    { 'bank': 'brass ensemble', 'audio_file': 'brass_ensemble_test.aiff', 'data_file':'brass_ensemble_test.txt'},
    { 'bank': 'percussion', 'audio_file': 'perc_test.aiff', 'data_file':'perc_test.txt'},
    { 'bank': 'string ensemble', 'audio_file': 'string_test.aiff', 'data_file':'string_test.txt'},
    { 'bank': 'square', 'audio_file': 'square_test.aiff', 'data_file':'square_test.txt'},
    { 'bank': 'triangle', 'audio_file': 'triangle_test.aiff', 'data_file':'triangle_test.txt'},
    { 'bank': 'saw', 'audio_file': 'saw_test.aiff', 'data_file':'saw_test.txt'},
    { 'bank': 'woodwind ensemble', 'audio_file': 'woodwind_test.aiff', 'data_file':'woodwind_test.txt'},
    { 'bank': 'clean guitar', 'audio_file': 'clean_guitar_test.aiff', 'data_file':'clean_guitar_test.txt'},
    { 'bank': 'pixelated guitar', 'audio_file': 'pixelated_guitar_test.aiff', 'data_file':'pixelated_guitar_test.txt'},
    #{ 'bank': 'vibraphone', 'audio_file': 'vibraphone_test.aiff', 'data_file':'vibraphone_test.txt'}
]

samples_hash = abs(hash(frozenset([frozenset(x.iteritems()) for x in samples_hyper_parameters])))
samples_file_name = '%s_Samples.p'%samples_hash



In [17]:
#All this assumes a one-second sample in the original file, otherwise
#the 150ms margins don't apply!

def get_sample(y,ts1,ts2,sr,chirp_ts,chirp_ind):
    ind1 = int(chirp_ind+(ts1+150-chirp_ts)*sr/1000.0)
    ind2 = int(chirp_ind+(ts2-150-chirp_ts)*sr/1000.0)
    return y[ind1:ind2]

def get_sample_by_meta(y,data,bank,pitch,vel,sr,chirp_ts,chirp_ind):
    ts1 = data.ts[(data.pitch==pitch) & (data.bank==bank) & (data.vel==vel)].iloc[0]
    ts2 = data.ts[np.where((data.pitch==pitch) & (data.bank==bank) & (data.vel==vel))[0]+1].iloc[0]
    ind1 = int(chirp_ind+(ts1+150-chirp_ts)*sr/1000.0)
    ind2 = int(chirp_ind+(ts2-150-chirp_ts)*sr/1000.0)
    return y[ind1:ind2]   

def get_fft(sample):
    try:
        if np.sum(np.abs(sample))<1.0: return []
        S_sample = np.abs((librosa.stft(sample)))**0.5
        tmp = np.abs(S_sample[:,S_sample.shape[1]/2])
        #Fix this so that it only returns if the FFT at the middle point crosses a threshold
        return tmp/np.sum(tmp)
    except:
        return []
    
def get_fft_with_pitch_shift(sample,sr,n_steps=0):
    if np.sum(np.abs(sample))<1.0: return []
    S_sample = np.abs((librosa.stft(sample)))**0.5
    S_sample = np.abs((librosa.stft(librosa.effects.pitch_shift(sample,sr,n_steps=n_steps))))**0.5
    tmp = np.abs(S_sample[:,S_sample.shape[1]/2])
    return tmp/np.sum(tmp)




In [18]:
def add_samples(audio_path,data_path,output_bank,samples):
    
    #output_bank refers to the bank in the final samples dataframe
    #for now only uses input bank '0', since I don't have the patience to load more banks
    
    
    print("Loading files for bank %s: %s and %s"%(output_bank,audio_path,data_path))

    y, sr = librosa.load(audio_path)
    print("Finished loading audio file")
    
    data = pd.read_csv(data_path, names=['ts', 'bank', 'pitch', 'vel'],sep=' ')
    print("Finished load data file")
    print(' ')
    
    chirp_ts = data.ts[data.bank=='start'].iloc[0]
    #print(chirp_ts)
    
    chirp_ind = np.min(np.where(y>0.1))
    #print(chirp_ind)
    
    print("Old Samples Shape")
    print(samples.shape)
    print(' ')
    
    print("Putting samples into DataFrame for bank %s"%(output_bank))
    
    for pitch in np.unique(data.pitch):
        for vel in np.setdiff1d(np.unique(data.vel),0):
            for bank in ('0',): #np.setdiff1d(np.unique(data.bank),'start'):
                if np.isnan(pitch): continue
                if np.isnan(vel): continue
                try:
                    sample = get_sample_by_meta(y,data,bank,int(pitch),int(vel),sr,chirp_ts,chirp_ind)
                    samples = samples.append({'bank':output_bank,'pitch':pitch,'vel':vel,'sample':sample,'sr':sr},ignore_index=True)
                except: continue
    
    print("New Samples Shape")
    print(samples.shape)
    print(' ')

    print("New Samples Head")
    print(samples.head())
    print(' ')
    
    print("New Samples Tail")
    print(samples.tail())
    print(' ')
    
    print("Success for bank %s!"%(output_bank))
    print(' ')
    
    return samples

def add_samples_wrapper(list_of_dicts,samples):
    for tmp_dict in list_of_dicts:
        print(tmp_dict)
        audio_path = tmp_dict['audio_file']
        data_path = tmp_dict['data_file']
        bank = tmp_dict['bank']
        samples = add_samples(audio_path,data_path,bank,samples)
        
    samples['fft'] = [get_fft(sample) for sample in samples['sample']]
    samples['total_sample'] = [sum(np.abs(sample)) for sample in samples['sample']]
    samples['total_fft'] = [sum(np.abs(sample)) for sample in samples['fft']]
    samples['valid_sample'] = [True if len(x)>0 else False for x in samples['fft']]

    #remove bad samples
    samples = samples[samples['valid_sample']]

    pickle.dump(samples,open(samples_file_name,'w'))
    print("Saved Samples to %s"%samples_file_name)
    print(' ')
    
    return samples

In [22]:
#Load the newest samples file
dated_files = [(os.path.getmtime(fn), os.path.basename(fn)) 
               for fn in os.listdir('.') if fn.endswith('_Samples.p')]
dated_files.sort()
dated_files.reverse()

if len(dated_files)==0:
    
    print("*sigh* you deleted the samples file or I've misplaced it")
    print("*sigh* okay, I'll make it then")
    print("*sigh* and save it to %s"%samples_file_name)
    print(" ")
    
    samples = pd.DataFrame()
    
    list_of_dicts = samples_hyper_parameters
    for tmp_dict in list_of_dicts:
        print(tmp_dict)
        audio_path = tmp_dict['audio_file']
        data_path = tmp_dict['data_file']
        bank = tmp_dict['bank']
        samples = add_samples(audio_path,data_path,bank,samples)
    
else:
    
    newest = dated_files[0][1]
    print("Loading Samples from %s"%newest)
    samples = pickle.load( open( newest, "rb" ) )
    print("Samples Loaded Here's The Size:")
    print(samples.shape)
    print("Here are the sample banks")
    present_banks = np.unique(samples['bank'])
    print(present_banks)
    print("Here are the missing banks")
    missing_banks = np.setdiff1d([x['bank'] for x in samples_hyper_parameters],present_banks)
    if len(missing_banks)==0: print("no missing banks!")
    else: print(missing_banks)
    print(" ")

    index = np.where([x['bank'] in missing_banks for x in samples_hyper_parameters])[0]
    list_of_dicts = [samples_hyper_parameters[index]]
        
    for tmp_dict in list_of_dicts:
        print(tmp_dict)
        audio_path = tmp_dict['audio_file']
        data_path = tmp_dict['data_file']
        bank = tmp_dict['bank']
        samples = add_samples(audio_path,data_path,bank,samples)
        
    

*sigh* you deleted the samples file or I've misplaced it
*sigh* okay, I'll make it then
*sigh* and save it to 3603226917487066209_Samples.p
 
{'data_file': 'piano_test.txt', 'audio_file': 'piano_test.aiff', 'bank': 'grand piano'}
Loading files for bank grand piano: piano_test.aiff and piano_test.txt
Finished loading audio file
Finished load data file
 
Old Samples Shape
(0, 0)
 
Putting samples into DataFrame for bank grand piano
New Samples Shape
(484, 5)
 
New Samples Head
          bank  pitch                                             sample  \
0  grand piano    1.0                                                 []   
1  grand piano    1.0  [-1.23435e-06, 1.74142e-06, -1.09021e-07, -4.0...   
2  grand piano    1.0  [-4.12916e-06, -1.99841e-07, 1.99631e-06, 4.10...   
3  grand piano    1.0  [5.91196e-06, -2.40699e-06, -2.87837e-06, 2.15...   
4  grand piano    2.0  [3.6356e-06, -1.16057e-06, -1.85886e-07, -1.01...   

        sr    vel  
0  22050.0   32.0  
1  22050.0   64.0  
2  

In [23]:

samples['fft'] = [get_fft(sample) for sample in samples['sample']]
samples['total_sample'] = [sum(np.abs(sample)) for sample in samples['sample']]
samples['total_fft'] = [sum(np.abs(sample)) for sample in samples['fft']]
samples['valid_sample'] = [True if len(x)>0 else False for x in samples['fft']]

#remove bad samples
samples = samples[samples['valid_sample']]

pickle.dump(samples,open(samples_file_name,'w'))
print("Saved Samples to %s"%samples_file_name)
print(' ')
    

Saved Samples to 3603226917487066209_Samples.p
 


# Test that Samples are the Correct Pitch (or at least align)

In [71]:
#Test there are no anomalies in the data
tmp2 = len(np.unique(samples['vel']))
for bank in np.unique(samples['bank']):
    for pitch in np.unique(samples['pitch']):
        tmp = len(np.where((samples['bank']==bank) & (samples['pitch']==pitch))[0])
        if tmp < tmp2: print ("Missing %s %g for %g"%(bank,pitch,tmp2-tmp))

Missing brass ensemble 1 for 4
Missing brass ensemble 2 for 4
Missing brass ensemble 3 for 4
Missing brass ensemble 4 for 4
Missing brass ensemble 5 for 4
Missing brass ensemble 6 for 4
Missing brass ensemble 7 for 4
Missing brass ensemble 8 for 4
Missing brass ensemble 9 for 4
Missing brass ensemble 10 for 4
Missing brass ensemble 11 for 4
Missing brass ensemble 12 for 4
Missing brass ensemble 13 for 4
Missing brass ensemble 14 for 4
Missing brass ensemble 15 for 4
Missing brass ensemble 16 for 4
Missing brass ensemble 17 for 4
Missing brass ensemble 18 for 4
Missing brass ensemble 19 for 4
Missing brass ensemble 20 for 4
Missing brass ensemble 21 for 4
Missing brass ensemble 22 for 4
Missing brass ensemble 23 for 4
Missing brass ensemble 24 for 4
Missing brass ensemble 91 for 4
Missing brass ensemble 92 for 4
Missing brass ensemble 93 for 4
Missing brass ensemble 94 for 4
Missing brass ensemble 95 for 4
Missing brass ensemble 96 for 4
Missing brass ensemble 97 for 4
Missing brass ens

In [24]:
for bank in np.unique(samples['bank']):
    vel = 128
    pitch = 62
    print(bank)
    index = np.where((samples['bank']==bank) & (samples['vel']==vel) & (samples['pitch']==pitch))[0]
    sample = np.array(samples.iloc[index]['sample'])[0]
    sr = float(samples.iloc[index]['sr'])
    IPython.display.display(IPython.display.Audio(data=sample, rate=sr))

brass ensemble


clean guitar


grand piano


percussion


pixelated guitar


saw


square


string ensemble


triangle


woodwind ensemble


# Create Data Frame of Pitch-Shifting At Every Cent a Half Semitone Above and Below

In [30]:
samples.head()

,bank,pitch,sample,sr,vel,fft,total_sample,total_fft,valid_sample
84,grand piano,22.0,"[-0.0180393, -0.0200819, -0.0218513, -0.023266...",22050.0,32.0,"[0.0108849, 0.00624392, 0.00514963, 0.00370402...",174.098040,1.0,True
85,grand piano,22.0,"[0.0129006, 0.0132442, 0.0145289, 0.0168158, 0...",22050.0,64.0,"[0.0100544, 0.00704283, 0.00254258, 0.00250298...",338.410861,1.0,True
86,grand piano,22.0,"[-0.0058777, -0.00111835, -0.000515375, -0.001...",22050.0,96.0,"[0.00803446, 0.00545483, 0.00265819, 0.0025026...",792.805898,1.0,True
87,grand piano,22.0,"[-0.274159, -0.235912, -0.186702, -0.140927, -...",22050.0,128.0,"[0.00766389, 0.0054533, 0.00105393, 0.00099795...",1641.952605,1.0,True
88,grand piano,23.0,"[-0.00744185, -0.00699556, -0.00639914, -0.005...",22050.0,32.0,"[0.0112511, 0.00964541, 0.00665629, 0.00472498...",166.695626,1.0,True


In [46]:
#make fft for pitch shifts once... no need to keep recalculating this stuff
ranges = np.multiply(range(-50,51,1),1.0/100.0)
print(ranges)
samples_with_pitch_shift_fft = pd.DataFrame()
print(len(ranges))

[-0.5  -0.49 -0.48 -0.47 -0.46 -0.45 -0.44 -0.43 -0.42 -0.41 -0.4  -0.39
 -0.38 -0.37 -0.36 -0.35 -0.34 -0.33 -0.32 -0.31 -0.3  -0.29 -0.28 -0.27
 -0.26 -0.25 -0.24 -0.23 -0.22 -0.21 -0.2  -0.19 -0.18 -0.17 -0.16 -0.15
 -0.14 -0.13 -0.12 -0.11 -0.1  -0.09 -0.08 -0.07 -0.06 -0.05 -0.04 -0.03
 -0.02 -0.01  0.    0.01  0.02  0.03  0.04  0.05  0.06  0.07  0.08  0.09
  0.1   0.11  0.12  0.13  0.14  0.15  0.16  0.17  0.18  0.19  0.2   0.21
  0.22  0.23  0.24  0.25  0.26  0.27  0.28  0.29  0.3   0.31  0.32  0.33
  0.34  0.35  0.36  0.37  0.38  0.39  0.4   0.41  0.42  0.43  0.44  0.45
  0.46  0.47  0.48  0.49  0.5 ]
101


In [59]:

for i in range(samples.shape[0]):
    sample_row = copy.copy(samples.iloc[i])
    #print(sample_row)
    sample = sample_row['sample']
    del sample_row['sample']
    sr = sample_row['sr']
    print("Bank %s, Pitch %g, Vel %g"%(sample_row['bank'],sample_row['pitch'],sample_row['vel']))
    if samples_with_pitch_shift_fft.shape[0]>0:
        index = np.where((samples_with_pitch_shift_fft['bank']==sample_row['bank']) &
          (samples_with_pitch_shift_fft['pitch']==sample_row['pitch'])&
          (samples_with_pitch_shift_fft['vel']==sample_row['vel']))
        len_matches = len(index[0])
        #print(len_matches)
        if samples_with_pitch_shift_fft.shape[0]>0 and len_matches==len(ranges): 
            print('skipped')
            continue
        elif len_matches>0:
            print('deleting incomplete part of dataframe')
            samples_with_pitch_shift_fft = samples_with_pitch_shift_fft.iloc[np.setdiff1d(range(samples_with_pitch_shift_fft.shape[0]),index)]
    for pitch_shift in ranges:
        #if pitch_shift*10==round(pitch_shift*10): print('Pitch shift: '+str(pitch_shift))
        new_sample_row = copy.copy(sample_row)
        new_sample_row['pitch_shift'] = pitch_shift
        new_sample_row['fft'] = get_fft_with_pitch_shift(sample,sr,n_steps=pitch_shift)
        samples_with_pitch_shift_fft = samples_with_pitch_shift_fft.append(new_sample_row)

Bank grand piano, Pitch 22, Vel 32
skipped
Bank grand piano, Pitch 22, Vel 64
skipped
Bank grand piano, Pitch 22, Vel 96
skipped
Bank grand piano, Pitch 22, Vel 128
skipped
Bank grand piano, Pitch 23, Vel 32
skipped
Bank grand piano, Pitch 23, Vel 64
skipped
Bank grand piano, Pitch 23, Vel 96
skipped
Bank grand piano, Pitch 23, Vel 128
skipped
Bank grand piano, Pitch 24, Vel 32
skipped
Bank grand piano, Pitch 24, Vel 64
skipped
Bank grand piano, Pitch 24, Vel 96
skipped
Bank grand piano, Pitch 24, Vel 128
skipped
Bank grand piano, Pitch 25, Vel 32
skipped
Bank grand piano, Pitch 25, Vel 64
deleting incomplete part of dataframe
Bank grand piano, Pitch 25, Vel 96
Bank grand piano, Pitch 25, Vel 128
Bank grand piano, Pitch 26, Vel 32
Bank grand piano, Pitch 26, Vel 64
Bank grand piano, Pitch 26, Vel 96
Bank grand piano, Pitch 26, Vel 128
Bank grand piano, Pitch 27, Vel 32
Bank grand piano, Pitch 27, Vel 64
Bank grand piano, Pitch 27, Vel 96
Bank grand piano, Pitch 27, Vel 128
Bank grand p

In [61]:
samples_with_pitch_shift_fft.head()



,bank,fft,pitch,pitch_shift,sr,total_fft,total_sample,valid_sample,vel
84,grand piano,"[0.0112701, 0.00735047, 0.00401337, 0.00249284...",22.0,-0.50,22050.0,1.0,174.09804,1.0,32.0
84,grand piano,"[0.01125, 0.00731963, 0.00404726, 0.00252299, ...",22.0,-0.49,22050.0,1.0,174.09804,1.0,32.0
84,grand piano,"[0.0112285, 0.00728791, 0.00408018, 0.00255324...",22.0,-0.48,22050.0,1.0,174.09804,1.0,32.0
84,grand piano,"[0.0112004, 0.00725215, 0.00411031, 0.00258255...",22.0,-0.47,22050.0,1.0,174.09804,1.0,32.0
84,grand piano,"[0.0111783, 0.00722049, 0.00414223, 0.00261384...",22.0,-0.46,22050.0,1.0,174.09804,1.0,32.0


In [67]:
samples_with_pitch_shift_fft_file_name = samples_file_name[:-10]+'_SamplesWithPitchShiftFFT.p'
pickle.dump(samples_with_pitch_shift_fft,open(samples_with_pitch_shift_fft_file_name,'w'))
print("Saved Samples to %s"%samples_with_pitch_shift_fft_file_name)
print(' ')



Saved Samples to 3603226917487066209_SamplesWithPitchShiftFFT.p
 


In [ ]:
#Load the newest samples file
dated_files = [(os.path.getmtime(fn), os.path.basename(fn)) 
               for fn in os.listdir('.') if fn.endswith('_SamplesWithPitchShiftFFT.p')]
dated_files.sort()
dated_files.reverse()
newest = dated_files[0][1]
print("Loading Samples with Pitch Shift FFT from %s"%newest)
samples = pickle.load( open( newest, "rb" ) )
print("Samples Loaded Here's The Size:")
print(samples.shape)
print("Here are the sample banks")
present_banks = np.unique(samples['bank'])
print(present_banks)
print(" ")
print("Here is a peek")
print(samples.head())
print(" ")
